In [1]:
!pip install -q git+https://github.com/MattiWe/ir_datasets.git@add-msmarco-v2.1-trec-rag

In [2]:
import pyterrier as pt
from datasets import load_dataset
import pandas as pd
import ir_datasets
import pyterrier_alpha as pta

/opt/miniconda3/envs/emalir-dr-rag/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
mmlu = load_dataset("cais/mmlu", "all", split="test")
# Creiamo un DataFrame con colonne qid e query
queries = []
for idx, ex in enumerate(mmlu):
    qid = f"{ex['subject']}_{idx}"
    queries.append({"qid": qid, "query": ex["question"]})
queries_df = pd.DataFrame(queries)

In [3]:
data = queries_df.rename(columns={"qid": "qid", "query": "text"})
data["text"] = data["text"].str.replace('\n', '\\n')
data.to_csv("./data/datasets/mmlu/mmlu-queries.tsv", sep="\t", index=False)

In [4]:
dataset = ir_datasets.load('msmarco-segment-v2.1')
pt_dataset = pt.get_dataset("irds:msmarco-segment-v2.1")

In [5]:
def rename_segment(run):
    run = run.rename(columns={"segment": "text"})
    return run
rename_pipe = pt.apply.generic(rename_segment)

In [6]:
df_run = pd.read_csv(
    "../data/processed/rag/old__setencoder-novelty-base__msmarco-segment-mmlu.tsv", sep="\t",
    names=["qid", "Q0", "docno", "rank", "score", "run_name"],
)

def get_ranking(run):
    return run
pta_run = pt.apply.generic(get_ranking)

pipe = pta_run >> pt.text.get_text(pt_dataset, "segment") >> rename_pipe 
results = pipe(df_run)


In [7]:
# salva i risultati nel TSV originario
results_top20 = results.groupby("qid", group_keys=False).head(20)

import re
def escape_text(s):
    # Sostituisci backslash, tab, newline, carriage return
    s = re.sub(r'\\', r'\\\\', s)
    s = re.sub(r'\t', ' ', s)
    s = re.sub(r'\r\n', r'\\n', s)
    s = re.sub(r'\r', r'\\n', s)
    s = re.sub(r'\n', r'\\n', s)
    # Sostituisci tutti i caratteri di controllo ASCII (0-31) con spazio
    s = re.sub(r'[\x00-\x1F]', ' ', s)
    return s

results_top20['text'] = results_top20['text'].apply(escape_text)
results_top20.to_csv(
    "../data/processed/rag/__setencoder-novelty-base__msmarco-segment-mmlu.tsv",
    sep="\t", index=False, header=False, 
)

/tmp/ipykernel_34055/2874647852.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results_top20['text'] = results_top20['text'].apply(escape_text)


In [17]:
index = pta.Artifact.from_hf('namawho/msmarco-segment-v2.1.pisa')
bm25_ret = index.bm25(verbose=True) % 100 >> pt.text.get_text(pt_dataset, "segment") >> rename_pipe

In [18]:
retrieved = bm25_ret.transform(queries_df)

PISA bm25: 100%|██████████| 14042/14042 [4:24:14<00:00,  1.13s/query]  


In [20]:
retrieved["Q0"] = "Q0"
retrieved["tag"] = "bm25"
retrieved = retrieved[["qid","Q0","docno","rank","score","tag"]]

# scrivi su file
retrieved.to_csv("bm25_mmlu.run", sep=" ", index=False, header=False)